### Thư viện cần thiết

In [45]:
import pandas as pd
import re
import torch
import torch.nn as nn
import math

### Đọc dữ liệu từ tệp txt

In [47]:
en_train_file = '/content/drive/MyDrive/Released Corpus/train.en.txt'
vi_train_file = '/content/drive/MyDrive/Released Corpus/train.vi.txt'

# Read lines
with open(en_train_file, 'r', encoding='utf-8') as f_train_en:
    en_train_lines = [line.strip() for line in f_train_en.readlines()]

with open(vi_train_file, 'r', encoding='utf-8') as f_train_vi:
    vi_train_lines = [line.strip() for line in f_train_vi.readlines()]

# Check whether length of en_train equal to length of vi_train
assert len(en_train_lines) == len(vi_train_lines)

train = pd.DataFrame({'en': en_train_lines, 'vi': vi_train_lines})

In [48]:
en_test_file = '/content/drive/MyDrive/Released Corpus/test.en.txt'
vi_test_file = '/content/drive/MyDrive/Released Corpus/test.vi.txt'

# Read lines
with open(en_test_file, 'r', encoding='utf-8') as f_test_en:
    en_test_lines = [line.strip() for line in f_test_en.readlines()]

with open(vi_test_file, 'r', encoding='utf-8') as f_test_vi:
    vi_test_lines = [line.strip() for line in f_test_vi.readlines()]

# Check whether length of en_train equal to length of vi_train
assert len(en_test_lines) == len(vi_test_lines)

test = pd.DataFrame({'en': en_test_lines, 'vi': vi_test_lines})

In [49]:
train

,en,vi
0,"To evaluate clinical, subclinical symptoms of ...","Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bện..."
1,"Evaluate clinical, subclinical symptoms of pat...","Đánh giá đặc điểm lâm sàng, cận lâm sàng bệnh ..."
2,There was a relation between vasodilatation an...,Có sự liên quan giữa độ quá phát V.a với mức đ...
3,Otittis media effusion on V a is a common dise...,Kết luận: Viêm tai ứ dịch trên viêm V.a là bện...
4,"Main symptoms are rhinitis, nasal congestion, ...","Triệu chứng cơ năng nổi bật là chảy mũi, ngạt ..."
...,...,...
499995,Patients and methods: Over-40-year-old men are...,Đối tượng và phương pháp nghiên cứu: Nam giới ...
499996,"If chronic, the age at onset (eg, since birth,...","Nếu mạn tính, cần xác định thời điểm xuất hiện..."
499997,Equipment for Removing a Tick Cleansing soluti...,Thiết bị dùng để lấy bọ ve Dung dịch rửa như d...
499998,Normal sigmoid at TVUS.,Đại tràng Sigma bình thưởng trên Siêu âm qua n...


In [50]:
test

,en,vi
0,"Knowledge, practices in public health service ...",Thực trạng kiến thức và thực hành của người có...
1,"Describe knowledge, practices in public health...","Mô tả thực trạng kiến thức, thực hành của ngườ..."
2,Methodology: A cross sectional study was used ...,Phương pháp: Thiết kế nghiên mô tả cắt ngang đ...
3,Results: Percentage of card's holders who knew...,Kết quả: Tỷ lệ người biết được khám chữa bệnh ...
4,Percentage of card's holders who went to the f...,Tỷ lệ người có thẻ BHYT thực hành khám chữa bệ...
...,...,...
2995,"Therefore, we conduct research to evaluate the...",Chính vì vậy chúng tôi tiến hành nghiên cứu đá...
2996,Methods: A cross-sectional descriptive study w...,Phương pháp nghiên cứu: Nghiên cứu mô tả cắt n...
2997,"Results: In 169 patients, 23.1% and 19.5% pati...",Số lượng bệnh nhân bị di căn hạch rốn phổi cao...
2998,Self-tanning products do not provide significa...,Các sản phẩm tự tạo màu da không có khả năng b...


### Hàm tiền xử lý dữ liệu cho cả văn bản tiếng Anh và tiếng Việt

In [51]:
def english_preprocessing(text):
    # Chuyển chữ hoa thành chữ thường
    text = text.lower()

    # Chuẩn hóa khoảng trắng ban đầu
    text = re.sub(r'\s+', ' ', text.strip())

    # Tách dấu hai chấm nếu dính liền (e.g., methods:This → methods: This)
    text = re.sub(r'(?<=\w):(?=\w)', ': ', text)

    # Xóa dấu ngoặc kép không cần thiết, nhưng giữ lại dấu nháy đơn trong từ (e.g., it's, don't)
    text = re.sub(r'[“”\"`]', '', text)

    # Xử lý đơn vị viết dính (e.g., 25ui/l → 25 ui/l)
    text = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', text)

    # Chuẩn hóa số: "81, 3%" → "81.3%", "9, 001" → "9001"
    text = re.sub(r'(\d),\s*(\d)', r'\1.\2', text)  # 81, 3 → 81.3
    text = re.sub(r'(?<=\d)\s*,\s*(?=\d)', '', text)  # 9, 001 → 9001

    # Chuẩn hóa các loại dash
    text = re.sub(r'[–—−]', '-', text)

    # Giữ lại định dạng đúng cho các ký hiệu y học
    text = re.sub(r'\s*/\s*', '/', text)   # PET / CT → PET/CT
    text = re.sub(r'\s*\+\s*', '+', text)  # ( + ) → (+)

    # Thêm khoảng trắng quanh toán tử
    text = re.sub(r'\s*(<=|>=|=|≠|±|<|>)\s*', r' \1 ', text)

    # Tách số và dấu %
    text = re.sub(r'(\d+(\.\d+)?)%', r'\1 %', text)

    # Làm sạch cuối cùng
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [52]:
def vietnamese_preprocessing(text):
    # Chuyển chữ hoa thành chữ thường
    text = text.lower()

    # Chuẩn hóa khoảng trắng ban đầu
    text = re.sub(r'\s+', ' ', text.strip())

    # Tách dấu hai chấm nếu dính liền (e.g., methods:This → methods: This)
    text = re.sub(r'(?<=\w):(?=\w)', ': ', text)

    # Loại bỏ dấu câu không cần thiết
    text = re.sub(r'[“”\"\'`]', '', text)

    # Xử lý đơn vị viết dính (e.g., 25ui/l → 25 ui/l)
    text = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', text)

    # Chuẩn hóa số: "81, 3%" → "81.3%", "9, 001" → "9001"
    text = re.sub(r'(\d),\s*(\d)', r'\1.\2', text)  # 81, 3 → 81.3
    text = re.sub(r'(?<=\d)\s*,\s*(?=\d)', '', text)  # 9, 001 → 9001

    # Chuẩn hóa các loại dash
    text = re.sub(r'[–—−]', '-', text)

    # Giữ lại định dạng đúng cho các ký hiệu y học
    text = re.sub(r'\s*/\s*', '/', text)   # PET / CT → PET/CT
    text = re.sub(r'\s*\+\s*', '+', text)  # ( + ) → (+)

    # Thêm khoảng trắng quanh toán tử
    text = re.sub(r'\s*(<=|>=|=|≠|±|<|>)\s*', r' \1 ', text)

    # Tách số và dấu %
    text = re.sub(r'(\d+(\.\d+)?)%', r'\1 %', text)

    # Làm sạch cuối cùng
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [53]:
# Tiền xử lý dữ liệu đối với tập train và tập test
train['en'] = train['en'].apply(english_preprocessing)
train['vi'] = train['vi'].apply(vietnamese_preprocessing)

test['en'] = test['en'].apply(english_preprocessing)
test['vi'] = test['vi'].apply(vietnamese_preprocessing)

In [54]:
from tokenizers import Tokenizer
tokenizer_en = Tokenizer.from_file("/content/drive/MyDrive/Released Corpus/tokenizer_en.json")
tokenizer_vi = Tokenizer.from_file("/content/drive/MyDrive/Released Corpus/tokenizer_vi.json")

In [55]:
MAX_LEN = 256  # giới hạn chiều dài token sequence

def encode_dataset(df, tokenizer_en, tokenizer_vi):
    bos_en = tokenizer_en.token_to_id("<s>")
    eos_en = tokenizer_en.token_to_id("</s>")
    bos_vi = tokenizer_vi.token_to_id("<s>")
    eos_vi = tokenizer_vi.token_to_id("</s>")

    def truncate(seq, max_len):
        return seq[:max_len] if len(seq) > max_len else seq

    def encode_pair(en_text, vi_text):
        # Encode English
        en_ids = tokenizer_en.encode(en_text).ids
        en_ids = truncate(en_ids, MAX_LEN - 2)  # trừ 2 vì thêm <s> và </s>
        en_input = [bos_en] + en_ids + [eos_en]

        # Encode Vietnamese (một lần)
        vi_ids = tokenizer_vi.encode(vi_text).ids
        vi_ids = truncate(vi_ids, MAX_LEN - 2)  # trừ 2 vì thêm <s> và </s>
        vi_full = [bos_vi] + vi_ids + [eos_vi]

        # Tách ra:
        vi_input = vi_full[:-1]  # bỏ </s>
        vi_target = vi_full[1:]  # bỏ <s>

        return en_input, vi_input, vi_target

    encoded = df.apply(lambda row: encode_pair(row['en'], row['vi']), axis=1)
    en_input, vi_input, vi_target = zip(*encoded)

    return pd.DataFrame({
        'en_input': en_input,
        'vi_input': vi_input,
        'vi_target': vi_target
    })

In [56]:
train_encoded = encode_dataset(train, tokenizer_en, tokenizer_vi)
test_encoded = encode_dataset(test, tokenizer_en, tokenizer_vi)

# Transformer architecture

In [57]:
class InputEmbeddings(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super(InputEmbeddings, self).__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    # (batch, seq_len) --> (batch, seq_len, d_model)
    # Multiply by sqrt(d_model) to scale the embeddings according to the paper
    return self.embedding(x) * math.sqrt(self.d_model)

In [58]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, seq_len: int, dropout: float):
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    # Create a matrix of shape (seq_len, d_model)
    pe = torch.zeros(seq_len, d_model)
    # Create a vector of shape (seq_len)
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
    # Create a vector of shape (d_model)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
    # Apply sine to even indices
    pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
    # Apply cosine to odd indices
    pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
    # Add a batch dimension to the positional encoding
    pe = pe.unsqueeze(0) # (1, seq_len, d_model)
    # Register the positional encoding as a buffer
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
    return self.dropout(x)

In [59]:
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self, d_model: int, h: int, dropout: float):
    super().__init__()
    self.d_model = d_model # Embedding vector size
    self.h = h # Number of heads

    self.d_k = d_model // h # Dimension of vector seen by each head
    self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
    self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
    self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
    self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k = query.shape[-1]

    # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
    attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      # Write a very low value (indicating -inf) to the positions where mask == 0
      attention_scores = attention_scores.masked_fill(mask == 0, torch.finfo(attention_scores.dtype).min)

    attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
    if dropout is not None:
      attention_scores = dropout(attention_scores)
    return torch.matmul(attention_scores, value), attention_scores

  def forward(self, q, k, v, mask):
    # q, k, v: (batch, seq_len, d_model)
    # mask: (batch, seq_len, seq_len)
    query = self.w_q(q)
    key = self.w_k(k)
    value = self.w_v(v)

    # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
    key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

    # Calculate attention using function we will define next
    x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

    # Combine all the heads together
    # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
    x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

    # Apply one final linear transformation
    return self.w_o(x)

In [60]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float):
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [61]:
class ResidualConnection(nn.Module):
  def __init__(self, features: int, dropout: float):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = nn.LayerNorm(features)

  def forward(self, x, sublayer):
    # Apply residual connection
    return x + self.dropout(sublayer(self.norm(x)))

In [62]:
class EncoderLayer(nn.Module):
    def __init__(self, features: int,
                 self_attention_block1: MultiHeadAttentionBlock,
                 self_attention_block2: MultiHeadAttentionBlock,
                 feed_forward_block: FeedForwardBlock,
                 dropout: float):
        super().__init__()
        self.self_attention_block1 = self_attention_block1
        self.self_attention_block2 = self_attention_block2
        self.feed_forward_block = feed_forward_block
        # 3 residual connections: SA1, SA2, FFN
        self.residual_connections = nn.ModuleList([
            ResidualConnection(features, dropout),
            ResidualConnection(features, dropout),
            ResidualConnection(features, dropout)
        ])

    def forward(self, x, src_mask):
        # First self-attention
        x = self.residual_connections[0](x, lambda x: self.self_attention_block1(x, x, x, src_mask))
        # Second self-attention
        x = self.residual_connections[1](x, lambda x: self.self_attention_block2(x, x, x, src_mask))
        # Feed-forward
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [63]:
class Encoder(nn.Module):
  def __init__(self, features: int, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = nn.LayerNorm(features)

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [64]:
class DecoderLayer(nn.Module):
  def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connections[2](x, self.feed_forward_block)
    return x

In [65]:
class Decoder(nn.Module):
  def __init__(self, features: int, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = nn.LayerNorm(features)

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

In [66]:
class ProjectionLayer(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
    return self.proj(x)

In [67]:
class Transformer(nn.Module):

  def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

    d_model = src_embed.d_model
    self.src_pos_norm = nn.LayerNorm(d_model)
    self.tgt_pos_norm = nn.LayerNorm(d_model)

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    src = self.src_pos_norm(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    tgt = self.tgt_pos_norm(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)

In [68]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block_1 = MultiHeadAttentionBlock(d_model, h, dropout)
        encoder_self_attention_block_2 = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderLayer(d_model, encoder_self_attention_block_1, encoder_self_attention_block_2, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N - 2):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderLayer(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    # weight typing
    projection_layer.proj.weight = tgt_embed.embedding.weight

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

# Training process

In [69]:
# !pip install tqdm
from tqdm import tqdm

### Tạo Dataloader, chia thành các batch để xử lý song song

In [70]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, df):
        self.src = df['en_input']
        self.tgt_in = df['vi_input']
        self.tgt_out = df['vi_target']

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        # Chỉ trả về tensor, không padding
        return {
            'src': torch.tensor(self.src.iloc[idx], dtype=torch.long),
            'tgt_in': torch.tensor(self.tgt_in.iloc[idx], dtype=torch.long),
            'tgt_out': torch.tensor(self.tgt_out.iloc[idx], dtype=torch.long)
        }

# Hàm collate_fn
def collate_fn(batch):
    pad_id_en = tokenizer_en.token_to_id("<pad>")
    pad_id_vi = tokenizer_vi.token_to_id("<pad>")

    src_batch = [item['src'] for item in batch]
    tgt_in_batch = [item['tgt_in'] for item in batch]
    tgt_out_batch = [item['tgt_out'] for item in batch]

    # pad_sequence sẽ tự động pad đến độ dài lớn nhất trong batch
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=pad_id_en)
    tgt_in_padded = pad_sequence(tgt_in_batch, batch_first=True, padding_value=pad_id_vi)
    tgt_out_padded = pad_sequence(tgt_out_batch, batch_first=True, padding_value=pad_id_vi)

    return {
        'src': src_padded,
        'tgt_in': tgt_in_padded,
        'tgt_out': tgt_out_padded
    }

In [71]:
train_dataset = TranslationDataset(train_encoded)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

test_dataset = TranslationDataset(test_encoded)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [72]:
# Hàm tạo mask
def create_mask(src, tgt, pad_id=tokenizer_en.token_to_id("<pad>")):
    # src: (batch, src_len), tgt: (batch, tgt_len)
    src_mask = (src != pad_id).unsqueeze(1).unsqueeze(2)  # (batch, 1, 1, src_len)
    tgt_pad_mask = (tgt != pad_id).unsqueeze(1).unsqueeze(2)  # (batch, 1, 1, tgt_len)
    tgt_len = tgt.size(1)
    tgt_sub_mask = torch.tril(torch.ones((tgt_len, tgt_len), device=tgt.device)).bool()  # (tgt_len, tgt_len)
    tgt_mask = tgt_pad_mask & tgt_sub_mask  # (batch, 1, tgt_len, tgt_len)
    return src_mask, tgt_mask

### Hàm huấn luyện trong 1 epoch

In [73]:
from torch.amp import autocast, GradScaler
scaler = GradScaler()

def train_one_epoch(model, dataloader, optimizer, loss_fn, device, scheduler=None):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for batch in progress_bar:
        src = batch['src'].to(device)
        tgt_input = batch['tgt_in'].to(device)
        tgt_output = batch['tgt_out'].to(device)

        src_mask, tgt_mask = create_mask(src, tgt_input)

        optimizer.zero_grad()

        with autocast(device_type=device.type):  # mixed precision context
            encoder_output = model.encode(src, src_mask)
            decoder_output = model.decode(encoder_output, src_mask, tgt_input, tgt_mask)
            output = model.project(decoder_output)

            loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))

        # backward and step using scaler
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        if scheduler is not None:
            scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    return total_loss / len(dataloader)

### Hàm ước tính loss trong quá trình training

In [74]:
@torch.no_grad()
def evaluate_one_epoch(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0

    progress_bar = tqdm(dataloader, desc="Evaluating", leave=False)

    for batch in progress_bar:
        src = batch['src'].to(device)
        tgt_input = batch['tgt_in'].to(device)
        tgt_output = batch['tgt_out'].to(device)

        src_mask, tgt_mask = create_mask(src, tgt_input)

        encoder_output = model.encode(src, src_mask)
        decoder_output = model.decode(encoder_output, src_mask, tgt_input, tgt_mask)
        output = model.project(decoder_output)

        loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))
        total_loss += loss.item()
        progress_bar.set_postfix(val_loss=loss.item())

    return total_loss / len(dataloader)

### Warm up + Inverse Square Root learning rate scheduler

In [75]:
class WarmupInverseSquareRootScheduler:
    def __init__(self, optimizer, d_model, warmup_steps):
        self.optimizer = optimizer
        self.warmup_steps = warmup_steps
        self.d_model = d_model
        self.step_num = 0

    def step(self):
        self.step_num += 1
        lr = self._get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

    def _get_lr(self):
        arg1 = self.step_num ** (-0.5)
        arg2 = self.step_num * (self.warmup_steps ** (-1.5))
        return (self.d_model ** -0.5) * min(arg1, arg2)

### Hàm Beam Search

In [76]:
def causal_mask(size):
    """
    Tạo causal mask: chỉ cho phép mỗi vị trí nhìn thấy các token trước đó (kể cả chính nó).
    Output shape: (1, 1, size, size)
    """
    return torch.tril(torch.ones(size, size)).unsqueeze(0).unsqueeze(1)  # (1, 1, size, size)

def beam_search_decode_batch_parallel(
    model, src, src_mask, tokenizer_vi, max_len, device, beam_size=4, alpha=0.6
):
    def get_token_id(tok, fallback=None):
        tid = tokenizer_vi.token_to_id(tok)
        if tid is None and fallback is not None:
            tid = tokenizer_vi.token_to_id(fallback)
        return tid

    with torch.no_grad():
        batch_size = src.size(0)

        sos_id = get_token_id("<s>", "[SOS]")
        eos_id = get_token_id("</s>", "[EOS]")
        pad_id = get_token_id("<pad>", "[PAD]")

        if sos_id is None or eos_id is None or pad_id is None:
            raise ValueError(f"Special tokens missing: sos={sos_id}, eos={eos_id}, pad={pad_id}")

        def length_penalty_fn(length, alpha):
            return ((5.0 + length) ** alpha) / ((5.0 + 1.0) ** alpha)

        # Encode input
        memory = model.encode(src, src_mask)
        memory = memory.repeat_interleave(beam_size, dim=0)
        src_mask = src_mask.repeat_interleave(beam_size, dim=0)

        seqs = torch.full((batch_size * beam_size, 1), sos_id, dtype=torch.long, device=device)
        scores = torch.zeros(batch_size * beam_size, device=device)
        finished_flags = torch.zeros(batch_size * beam_size, dtype=torch.bool, device=device)

        for step in range(1, max_len):
            tgt_mask = causal_mask(seqs.size(1)).to(device)
            dec_out = model.decode(memory, src_mask, seqs, tgt_mask)
            logits = model.project(dec_out[:, -1, :])
            log_probs = torch.log_softmax(logits, dim=-1)

            # Beam đã kết thúc chỉ sinh <pad>
            log_probs[finished_flags] = -1e9
            log_probs[finished_flags, pad_id] = 0

            next_scores, next_tokens = torch.topk(log_probs, beam_size, dim=-1)

            # Reshape để chọn top beam cho batch
            next_scores = next_scores.view(batch_size, beam_size, beam_size)
            next_tokens = next_tokens.view(batch_size, beam_size, beam_size)

            # Tính total score với length penalty sớm
            total_scores = scores.view(batch_size, beam_size, 1) + next_scores
            lp = length_penalty_fn(step + 1, alpha)
            total_scores = total_scores / lp

            # Chọn top beam
            top_scores, top_indices = torch.topk(total_scores.view(batch_size, -1), beam_size, dim=-1)
            beam_indices = top_indices // beam_size
            token_indices = top_indices % beam_size

            # Vector hóa update sequences
            old_beam_ids = (beam_indices + torch.arange(batch_size, device=device).unsqueeze(1) * beam_size).view(-1)
            chosen_tokens = next_tokens[torch.arange(batch_size, device=device).unsqueeze(1), beam_indices, token_indices].view(-1)

            seqs = torch.cat([seqs[old_beam_ids], chosen_tokens.unsqueeze(1)], dim=-1)
            scores = top_scores.view(-1)

            # Cập nhật finished flags
            finished_flags = finished_flags[old_beam_ids] | (chosen_tokens == eos_id)

            if finished_flags.all():
                break

        # Chọn best beam cuối cùng
        final_seqs = []
        for b in range(batch_size):
            start = b * beam_size
            end = start + beam_size
            cand_scores = scores[start:end]
            cand_seqs = seqs[start:end]
            lengths = torch.tensor([len(s) for s in cand_seqs], dtype=torch.float, device=device)
            lp = length_penalty_fn(lengths, alpha)
            best_idx = torch.argmax(cand_scores / lp).item()
            final_seqs.append(cand_seqs[best_idx])

        # Pad output
        max_len_final = max(len(s) for s in final_seqs)
        padded = torch.full((batch_size, max_len_final), pad_id, dtype=torch.long, device=device)
        for i, seq in enumerate(final_seqs):
            padded[i, :len(seq)] = seq

        return padded

### Hàm ước tính BLEU trên tập dữ liệu

In [77]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

def evaluate_bleu(model, dataloader, tokenizer_en, tokenizer_vi, device, max_len=128):
    model.eval()
    references = []
    hypotheses = []

    smoothie = SmoothingFunction().method4

    sos_id = tokenizer_vi.token_to_id("[SOS]") or tokenizer_vi.token_to_id("<s>")
    eos_id = tokenizer_vi.token_to_id("[EOS]") or tokenizer_vi.token_to_id("</s>")
    pad_id = tokenizer_vi.token_to_id("<pad>")
    pad_src_id = tokenizer_en.token_to_id("<pad>")

    for batch in tqdm(dataloader, desc="Evaluating BLEU"):
        src = batch['src'].to(device)
        tgt_input = batch['tgt_in'].to(device)
        tgt_output = batch['tgt_out'].to(device)

        src_mask = (src != pad_src_id).unsqueeze(1).unsqueeze(2)

        pred_ids_batch = beam_search_decode_batch_parallel(model, src, src_mask, tokenizer_vi, max_len, device, beam_size=4)

        for pred_ids, ref_ids in zip(pred_ids_batch, tgt_output):
            pred_tokens = [
                tokenizer_vi.id_to_token(id.item())
                for id in pred_ids
                if id.item() not in {sos_id, eos_id, pad_id}
            ]

            ref_tokens = [
                tokenizer_vi.id_to_token(id.item())
                for id in ref_ids
                if id.item() not in {sos_id, eos_id, pad_id}
            ]

            hypotheses.append(pred_tokens)
            references.append([ref_tokens])  # each reference must be a list of lists

    bleu = corpus_bleu(references, hypotheses, smoothing_function=smoothie)

    print(f"BLEU score (nltk): {bleu * 100:.2f}")
    return bleu

### Load lại model với trọng số đã huấn luyện được lưu trong tệp 're_transformer_best.pth'

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

src_vocab_size = tokenizer_en.get_vocab_size()
tgt_vocab_size = tokenizer_vi.get_vocab_size()

# Load lại model
model = build_transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    src_seq_len=256,
    tgt_seq_len=256,
    d_model=512,
    N=6,
    h=8,
    dropout=0.1,
    d_ff=1024
).to(device)

checkpoint_path = '/content/drive/MyDrive/Released Corpus/re_transformer_best.pth'
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

### Kết quả BLEU trên tập test : 57.39

In [79]:
evaluate_bleu(model, test_dataloader, tokenizer_en, tokenizer_vi, device)

Evaluating BLEU: 100%|██████████| 94/94 [11:04<00:00,  7.07s/it]


BLEU score (nltk): 57.39


0.5738936505674651

### Hàm dịch câu tiếng Anh sang tiếng Việt

In [81]:
def translate_to_vietnamese(sentence_en: str) -> str:
    model.eval()
    device = next(model.parameters()).device

    # Tiền xử lý tiếng Anh
    src_text = english_preprocessing(sentence_en)
    src_ids = tokenizer_en.encode(src_text).ids
    src_ids = [tokenizer_en.token_to_id("<s>")] + src_ids + [tokenizer_en.token_to_id("</s>")]

    src_tensor = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)

    pad_id = tokenizer_en.token_to_id("<pad>")
    assert pad_id is not None, "Tokenizer English chưa có token <pad>"
    src_mask = (src_tensor != pad_id).unsqueeze(1).unsqueeze(2)

    with torch.no_grad():
        pred_ids = beam_search_decode_batch_parallel(
            model, src_tensor, src_mask, tokenizer_vi,
            max_len=128, device=device, beam_size=4, alpha=0.6
        )[0]

    pred_ids = pred_ids.cpu().numpy()

    # Dùng decode của tokenizer để ghép subword đúng chuẩn
    return tokenizer_vi.decode(pred_ids, skip_special_tokens=True)

### Code để chạy dịch máy trên tập test 3000 câu tiếng Anh

Kết quả được lưu trong tệp Re-transformer-test-translation.csv

In [83]:
import tqdm
vi_output = []

# Dịch toàn bộ 3000 câu tiếng Anh
for sentence in tqdm.tqdm(test['en'], desc="Translating test set"):
    translation = translate_to_vietnamese(sentence)
    vi_output.append(translation)

# Tạo dataframe mới
df_result = pd.DataFrame({
    'en': test['en'],          # câu tiếng Anh gốc
    'vi_output': vi_output,    # câu dịch từ model
    'vi_truth': test['vi']     # câu ground truth
})

# Lưu vào CSV
df_result.to_csv("Re-transformer-test-translation.csv", index=False, encoding='utf-8-sig')

Translating test set: 100%|██████████| 3000/3000 [12:01<00:00,  4.16it/s]
